### Import các thư viện cần thiết

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd '/content/drive/MyDrive/CS232.N21 Đồ án tính toán đa phương tiện'
%ls

Mounted at /content/drive
/content/drive/MyDrive/CS232.N21 Đồ án tính toán đa phương tiện
AdaptiveHuffman.py  code.ipynb  data/  index.py  Utils.py


## Thuật toán Arithmetic Coding

In [ ]:
import sys

In [ ]:
#Import các thư viện cần thiết
from collections import defaultdict
from fractions import Fraction
import decimal
from decimal import Decimal
from time import time
import os
import sys

#Tính toán khoảng xác xuất cho từng ký tự đầu vào của chuỗi
def calculate_probabilities(text):
    #Tạo dict với giá trị mặc định là 0
    frequencies = defaultdict(int)
    #Duyệt qua từng ký tự trong chuỗi và tăng tần xuất xuất hiện của ký tự trong dict
    for char in text:
        frequencies[char] += 1

    total_chars = len(text)
    probabilities = {}
    cumulative_prob = Fraction(0, 1) #Xác xuất tích lũy của ký tự

    #Tính toán khoảng xác xuất cho từng ký tự đầu vào của chuỗi
    for char, freq in frequencies.items():
        probability = Fraction(freq, total_chars)
        probabilities[char] = (cumulative_prob, cumulative_prob + probability)
        cumulative_prob += probability

    return probabilities

#Encode
def encode_text(text, probabilities):
    lower = Fraction(0, 1)
    upper = Fraction(1, 1)
    range_width = Fraction(1, 1) #Độ rộng của khoảng xác xuất
    start_time = time()
    #Duyệt qua từng ký tự trong chuỗi
    for char in text:
        range_start, range_end = probabilities[char] #Lấy khoảng giá trị (range_start, range_end) tương ứng từ probabilities của ký tự.
        upper = lower + range_width * range_end
        lower = lower + range_width * range_start
        range_width = upper - lower
    end_time = time()
    execution_time = end_time - start_time

    print("Thời gian mã hóa:", execution_time)
    return (lower+upper)/2

#Decode
def decode_text(encoded_value, probabilities, text_length):
    decoded_text = ""
    value = encoded_value

    for _ in range(text_length):
        for char, (range_start, range_end) in probabilities.items():
            range_width = range_end - range_start
            #Kiểm tra xem giá trị đã nén (value) có nằm trong khoảng xác suất của ký tự hiện tại hay không.
            if range_start <= value < range_start + range_width:
                decoded_text += char
                value = (value - range_start) / range_width
                break

    return decoded_text

# Example usage
text = input('Nhập một câu:')

probabilities = calculate_probabilities(text)
encoded_value = encode_text(text, probabilities)
encoded_value_float = float(encoded_value)
decoded_text = decode_text(encoded_value, probabilities, len(text))

print("Original Text:", text)
print("Encoded Value:", encoded_value_float)
print("Decoded Text:", decoded_text)
if text == decoded_text:
  print("giống")

Nhập một câu:a
Thời gian mã hóa: 2.9325485229492188e-05
Original Text: a
Encoded Value: 0.5
Decoded Text: a
giống


In [ ]:
print("Original text length:", len(text))
print("Original text size:", sys.getsizeof(text))
print("Arithmetic Coding compressed length:", sys.getsizeof(encoded_value))
print("Arithmetic Coding compressed length 2:", sys.getsizeof(probabilities))

Original text length: 21
Original text size: 70
Arithmetic Coding compressed length: 48
Arithmetic Coding compressed length 2: 640


## Thuật toán Adaptive Huffman

In [ ]:
import math
def get_freq(text):
    freq = dict()
    for c in text:
        if c in freq:
            freq[c] += 1
        else:
            freq[c] = 1
    freq = dict(sorted([(a,freq[a]) for a in freq if freq[a]>0.0], key = lambda el: el[1], reverse = True))
    Nin = sum([freq[a] for a in freq])
    freq = dict([(a,freq[a]/Nin) for a in freq])
    return freq
def encoded_text( text,code):
    encoded = ''
    for char in text:
        encoded += code[char]
    return encoded
def decoded_text(code, encoded):
    decoded = ''
    temp = ''
    for bit in encoded:
        temp += bit
        for char in code:
            if code[char] == temp:
                decoded += char
                temp = ''
    return decoded

def write_to_file(filename, text):
    f = open(filename, 'w+')
    f.write(text)
    f.close()
def read_from_file(filename):
    f = open(filename, 'r')
    text = f.read()
    f.close()
    return text

In [ ]:
# from Utils import *
from time import time
# đại diện cho một nút trong cây, bao gồm các thuộc tính cha, con trái, con phải, trọng số và ký tự
class Node(object):
    def __init__(self, parent=None, left=None, right=None, weight=0, symbol=''):
        super(Node, self).__init__()
        self._parent = parent
        self._left = left
        self._right = right
        self._weight = weight
        self._symbol = symbol
    @property
    def parent(self):
        return self._parent
    @parent.setter
    def parent(self, parent):
        self._parent = parent
    @property
    def left(self):
        return self._left
    @left.setter
    def left(self, left):
        self._left = left
    @property
    def right(self):
        return self._right
    @right.setter
    def right(self, right):
        self._right = right
    @property
    def weight(self):
        return self._weight
    @weight.setter
    def weight(self, weight):
        self._weight = weight
    @property
    def symbol(self):
        return self._symbol
    @symbol.setter
    def symbol(self, symbol):
        self._symbol = symbol

class AdaptiveHuffman(object):
    def __init__(self):
        super(AdaptiveHuffman, self).__init__()
        self.NYT = Node(symbol="NYT") # nút NYT
        self.root = self.NYT          # nút gốc
        self.nodes = []             # danh sách các nút trong cây
        self.seen = [None] * 256    # mảng để theo dôi các ký tự đã xuất hiện

    def get_code(self, s, node, code=''):
        if node.left is None and node.right is None:  # nếu nút là lá
            return code if node.symbol == s else ''   # kiểm tra nếu ký tự của nút là ký tự s đang xét, trả về mã code, ngược lại trả về chuỗi rỗng
        else:  # nếu nút không phải  là lá
            temp = ''
            if node.left is not None:
                temp = self.get_code(s, node.left, code+'0')
            if not temp and node.right is not None:
                temp = self.get_code(s, node.right, code+'1')
            return temp

    # tìm nút có trọng số lớn nhất
    def find_largest_node(self, weight):
        for n in reversed(self.nodes):
            if n.weight == weight:
                return n

    # hoán đổi vị trí giữa hai nút
    def swap_node(self, n1, n2):
        i1, i2 = self.nodes.index(n1), self.nodes.index(n2)
        self.nodes[i1], self.nodes[i2] = self.nodes[i2], self.nodes[i1]
        tmp_parent = n1.parent
        n1.parent = n2.parent
        n2.parent = tmp_parent
        if n1.parent.left is n2:
            n1.parent.left = n1
        else:
            n1.parent.right = n1

        if n2.parent.left is n1:
            n2.parent.left = n2
        else:
            n2.parent.right = n2

    # thêm ký tự vào cây Adaptive Huffman
    def insert(self, s):
        node = self.seen[ord(s)]  # kiểm tra ký tự đã xuất hiện trước đó hay chưa
        if node is None:  # nếu ký tự chưa xuất hiện trước đó
            spawn = Node(symbol=s, weight=1)
            internal = Node(symbol='', weight=1, parent=self.NYT.parent,
                            left=self.NYT, right=spawn)
            spawn.parent = internal
            self.NYT.parent = internal
            if internal.parent is not None:
                internal.parent.left = internal
            else:
                self.root = internal
            self.nodes.insert(0, internal)
            self.nodes.insert(0, spawn)
            self.seen[ord(s)] = spawn
            node = internal.parent

        while node is not None:  # nếu ký tự đã xuất hiện
            largest = self.find_largest_node(node.weight)
            if (node is not largest and node is not largest.parent and
                    largest is not node.parent):
                self.swap_node(node, largest)
            node.weight = node.weight + 1
            node = node.parent

    # encode
    def encode(self, text):
        result = ''
        for s in text:  # duyệt qua từng ký tự
            if self.seen[ord(s)]:   # nếu ký tự đã xuất hiện trước đó
                result += self.get_code(s, self.root)
            else:                   # nếu ký tự chưa xuất hiện
                result += self.get_code('NYT', self.root)
                result += bin(ord(s))[2:].zfill(8)
            self.insert(s)
        return result


    def get_ascii(self, bin_str):
        return chr(int(bin_str, 2))

    # decode
    def decode(self, text):
        result = ''

        symbol = self.get_ascii(text[:8]) # giải mã 8bit đầu tiên
        result += symbol
        self.insert(symbol) # cập nhật cây
        node = self.root

        # duyệt qua các bit từ vị trí thứ 8
        i = 8
        while i < len(text):
            node = node.left if text[i] == '0' else node.right
            symbol = node.symbol

            if symbol:
                if symbol == 'NYT':
                    symbol = self.get_ascii(text[i+1:i+9])
                    i += 8
                result += symbol
                self.insert(symbol)
                node = self.root
            i += 1
        return result
# if __name__ == '__main__':
# choice = 2 #('Chọn (1/2) :\n1. Mã hoá file văn bản\n2. Mã hoá xâu văn bản\n'))
# if choice == 1:
#     file = input('Nhập đường dẫn file: ')
#     text = read_from_file(file)
# else:
#     text = str(input('Nhập xâu cần mã hoá: '))
# start = time()
# encoded = AdaptiveHuffman().encode(text)
# end = time() - start
# print("Thời gian mã hóa là: ", end)
# start = time()
# decoded = AdaptiveHuffman().decode(encoded)
# end = time() - start
# print("Thời gian giải mã là: ", end)

## Thực nghiệm

### Đánh giá Arithmetic Coding

In [ ]:
# đánh giá Arithmetic Coding
file = "./data/test1.txt"
text = read_from_file(file)
# encode
probabilities = calculate_probabilities(text)
encoded_value = encode_text(text, probabilities)
encoded_value_float = float(encoded_value)
# decode
start = time()
decoded_text = decode_text(encoded_value, probabilities, len(text))
end = time() - start

if text == decoded_text:
  print("GIải mã đúng")
print("Thời gian giải mã là: ", end)
print("Original Text:", text)
print("Encoded Value:", encoded_value_float)
print("Decoded Text:", decoded_text)

print("Data file size:", sys.getsizeof(text))
compressed_size = sys.getsizeof(encoded_value)+sys.getsizeof(probabilities)
print("Arithmetic Coding compressed size:", compressed_size)
print("Compession ratio: {:.2f} ".format(1/((compressed_size)/(sys.getsizeof(text)))))
print("Compession performance: {:.2%} ".format((1 - (compressed_size/(sys.getsizeof(text))))))

Thời gian mã hóa: 7.2730841636657715
GIải mã đúng
Thời gian giải mã là:  1.0699923038482666
Original Text: KQggGwjisGOuSaUUcfyymfzqVWHgJenQusWmAZIlxLVQhHSXUCoSjAzCTsbzMilHTKyUEpkiPVxsTHqFHjITluhLFyVgZXYgNHEIlHeNqtwfOqeIffUvxQBiLFKSRALCkuYQtVEZVvjYOJTKwLczzoDyvdEGmhBTYAwzRkaFEaHorsTVEvpWFFJdwnlsgexCxQUOZrTKAtGnicPhJKTEsBqTvGSAwUYoFLidQlqfiRhTGUzXbxLZPNwIpflCEpJWaLsQdacwjUSDPeLrEPQESYvlyGImgxSfVDsXWtGppvkMFDQoSkIZBomoFERciAvHskqgFaQhAOjNrMMNakPaDohXHwmHGkJomgnuZfQVJMgzwuAORumgdohShtdEedxRuuCLKVRgqTvSTJqvkOhySapiFnNfVhLbQQVkSFVvRKDWcAqVKFHgwrcrpBxZxVlmxFbpzapGuMGSTOcKvOCreNDCOTOXkfptgkTVxbGxgIScEazGiKenBpkeGppepaxdjBYqTzFnaPBPvETiuwqTjqcGPljYpEkXtZaOnRcZXUHAOCuaYNKQQwStVZxCuUENqCiJktVDstrEmwRGubeWGXaTNENfCAUvmbihWmHTtNQcpNxdmuXFbTNjfbQcBWkHLBwNHVDnpEOUlXibLcurHTVmGtcqoZljKbnFKXRBRJekLWpZgAnqUJkdHwbujwQPFoDQoZFGCDObzdzsZKTGAOrwmqDiSlmfVTTtgLdQsMfEVovoulqvrdtRoCWcnMMKvuBVTnmnpUZQVqFOsLvosbbtgOUIporulduLaGsdWoyAIFkCnuiqbkFmMmhXeACKGIVMSSGsjDWzxbeJiwbOUYpPFxMITELbVrcEoNHPCEvZPnFryMXRLYekZZxTQ

In [ ]:
# Đánh giá Arithmetic Coding tổng hợp
for i in range(1,7):
  file = "./data/test"+str(i)+".txt"
  print("test"+str(i))
  text = read_from_file(file)
  # encode
  probabilities = calculate_probabilities(text)
  encoded_value = encode_text(text, probabilities)
  encoded_value_float = float(encoded_value)
  # decode
  start = time()
  decoded_text = decode_text(encoded_value, probabilities, len(text))
  end = time() - start

  if text == decoded_text:
    print("GIải mã đúng")
  print("Thời gian giải mã là: ", end)
  # print("Original Text:", text)
  # print("Encoded Value:", encoded_value_float)
  # print("Decoded Text:", decoded_text)

  print("Data file size:", sys.getsizeof(text))
  compressed_size = sys.getsizeof(encoded_value)+sys.getsizeof(probabilities)
  print("Arithmetic Coding compressed size:", compressed_size)
  print("Compession ratio: {:.2f} ".format(1/((compressed_size)/(sys.getsizeof(text)))))
  print("Compession performance: {:.2%} ".format((1 - (compressed_size/(sys.getsizeof(text))))))

test1
Thời gian mã hóa: 7.111246109008789
GIải mã đúng
Thời gian giải mã là:  0.8726646900177002
Data file size: 3049
Arithmetic Coding compressed size: 2320
Compession ratio: 1.31 
Compession performance: 23.91% 
test2
Thời gian mã hóa: 48.641120195388794
GIải mã đúng
Thời gian giải mã là:  3.9367563724517822
Data file size: 6049
Arithmetic Coding compressed size: 2320
Compession ratio: 2.61 
Compession performance: 61.65% 
test3
Thời gian mã hóa: 857.2591881752014
GIải mã đúng
Thời gian giải mã là:  13.842998266220093
Data file size: 15049
Arithmetic Coding compressed size: 2320
Compession ratio: 6.49 
Compession performance: 84.58% 
test4
Thời gian mã hóa: 920.5838882923126
GIải mã đúng
Thời gian giải mã là:  15.687084436416626
Data file size: 15049
Arithmetic Coding compressed size: 2320
Compession ratio: 6.49 
Compession performance: 84.58% 
test5
Thời gian mã hóa: 883.935079574585
GIải mã đúng
Thời gian giải mã là:  21.76703381538391
Data file size: 15049
Arithmetic Coding compre

In [ ]:
for i in range(4,5):
  file = "./data/test"+str(i)+".txt"
  print("test"+str(i))
  text = read_from_file(file)
  # encode
  probabilities = calculate_probabilities(text)
  encoded_value = encode_text(text, probabilities)
  encoded_value_float = float(encoded_value)
  # decode
  start = time()
  decoded_text = decode_text(encoded_value, probabilities, len(text))
  end = time() - start

  if text == decoded_text:
    print("GIải mã đúng")
  print("Thời gian giải mã là: ", end)
  # print("Original Text:", text)
  # print("Encoded Value:", encoded_value_float)
  # print("Decoded Text:", decoded_text)

  print("Data file size:", sys.getsizeof(text))
  compressed_size = sys.getsizeof(encoded_value)+sys.getsizeof(probabilities)
  print("Arithmetic Coding compressed size:", compressed_size)
  print("Compession ratio: {:.2f} ".format(1/((compressed_size)/(sys.getsizeof(text)))))
  print("Compession performance: {:.2%} ".format((1 - (compressed_size/(sys.getsizeof(text))))))

test4
Thời gian mã hóa: 281.34933280944824
GIải mã đúng
Thời gian giải mã là:  10.526010513305664
Data file size: 10049
Arithmetic Coding compressed size: 4744
Compession ratio: 2.12 
Compession performance: 52.79% 


In [ ]:
# Đánh giá Arithmetic Coding tổng hợp ver 2 + white space
for i in range(7,11):
  file = "./data/test"+str(i)+".txt"
  print("test"+str(i))
  text = read_from_file(file)
  # encode
  probabilities = calculate_probabilities(text)
  encoded_value = encode_text(text, probabilities)
  encoded_value_float = float(encoded_value)
  # decode
  start = time()
  decoded_text = decode_text(encoded_value, probabilities, len(text))
  end = time() - start

  if text == decoded_text:
    print("GIải mã đúng")
  print("Thời gian giải mã là: ", end)
  # print("Original Text:", text)
  # print("Encoded Value:", encoded_value_float)
  # print("Decoded Text:", decoded_text)

  print("Data file size:", sys.getsizeof(text))
  compressed_size = sys.getsizeof(encoded_value)+sys.getsizeof(probabilities)
  print("Arithmetic Coding compressed size:", compressed_size)
  print("Compession ratio: {:.2f} ".format(1/((compressed_size)/(sys.getsizeof(text)))))
  print("Compession performance: {:.2%} ".format((1 - (compressed_size/(sys.getsizeof(text))))))

test7
Thời gian mã hóa: 2.785757303237915
GIải mã đúng
Thời gian giải mã là:  0.8615086078643799
Data file size: 2049
Arithmetic Coding compressed size: 4744
Compession ratio: 0.43 
Compession performance: -131.53% 
test8
Thời gian mã hóa: 268.3152434825897
GIải mã đúng
Thời gian giải mã là:  11.525758266448975
Data file size: 10048
Arithmetic Coding compressed size: 4744
Compession ratio: 2.12 
Compession performance: 52.79% 
test9
Thời gian mã hóa: 66.88906574249268
GIải mã đúng
Thời gian giải mã là:  5.210064888000488
Data file size: 6048
Arithmetic Coding compressed size: 4744
Compession ratio: 1.27 
Compession performance: 21.56% 
test10
Thời gian mã hóa: 269.1017904281616
GIải mã đúng
Thời gian giải mã là:  11.382839918136597
Data file size: 10049
Arithmetic Coding compressed size: 4744
Compession ratio: 2.12 
Compession performance: 52.79% 


### Đánh giá Adaptive Huffman

In [ ]:
# Đánh giá Adaptive Huffman tổng hợp
for i in range(1,7):
  file = "./data/test"+str(i)+".txt"
  print("test"+str(i))
  text = read_from_file(file)
  start = time()
  encoded = AdaptiveHuffman().encode(text)
  end = time() - start
  print("Thời gian mã hóa là: ", end)
  start = time()
  decoded = AdaptiveHuffman().decode(encoded)
  end = time() - start
  print("Thời gian giải mã là: ", end)
  # display_(text, encoded, decoded)
  print("Data file size:", sys.getsizeof(text))
  print("Adaptive Huffman compressed length:", sys.getsizeof(encoded)/8)
  print("Compession ratio: {:.2f} ".format(1/(sys.getsizeof(encoded)/8/(sys.getsizeof(text)))))
  print("Compession performance: {:.2%} ".format((1 - (sys.getsizeof(encoded)/8/(sys.getsizeof(text))))))

test1
Thời gian mã hóa là:  0.18277692794799805
Thời gian giải mã là:  0.08566546440124512
Data file size: 3049
Adaptive Huffman compressed length: 2236.75
Compession ratio: 1.36 
Compession performance: 26.64% 
test2
Thời gian mã hóa là:  0.3363513946533203
Thời gian giải mã là:  0.1548750400543213
Data file size: 6049
Adaptive Huffman compressed length: 4401.5
Compession ratio: 1.37 
Compession performance: 27.24% 
test3
Thời gian mã hóa là:  0.8532841205596924
Thời gian giải mã là:  0.40928101539611816
Data file size: 15049
Adaptive Huffman compressed length: 10912.75
Compession ratio: 1.38 
Compession performance: 27.49% 
test4
Thời gian mã hóa là:  0.9993317127227783
Thời gian giải mã là:  0.42934203147888184
Data file size: 10049
Adaptive Huffman compressed length: 8431.5
Compession ratio: 1.19 
Compession performance: 16.10% 
test5
Thời gian mã hóa là:  1.4326744079589844
Thời gian giải mã là:  1.263484239578247
Data file size: 15049
Adaptive Huffman compressed length: 12565.875

In [ ]:
# Tạo tập dữ liệu đánh giá
import random
import string

def generate_random_string(length):
    letters = string.ascii_letters + string.digits + string.punctuation #+ string.whitespace
    random_string = ''.join(random.choice(letters) for _ in range(length))
    return random_string

def save_to_file(text, filename):
    with open(filename, 'w') as file:
        file.write(text)

# Nhập kích thước chuỗi ký tự ngẫu nhiên
length = 10000

# Tạo chuỗi ký tự ngẫu nhiên
random_string = generate_random_string(length)
print(random_string)
# Lưu chuỗi ký tự ngẫu nhiên vào file
filename = "./data/test4.txt"
save_to_file(random_string, filename)

# print(f"Random string of length {length} has been saved to {filename}.")


0?zGE1n^/j~e*w"Z~fA)b$W%OHmU";C"_(E5ZA4Fu9j23{JlLNj7Isr(R"3Ao]RdzCG!L'!)}0&xC7?1,"loj1rRW:wRy%yGNDB8d3P,.!["LG=$#~j!$a]#s"Z|i?/L's<s,uhO>s9@Io-}w(2Y;:{Vt2E}s.ar>a[7lAM]Lh`O>V:l<ruB.\n#?LkZ[.sEywK|U5n`JEwWq[j9T,fG~U3yq`]jc(+UB68OAnEm=TJ[(2'Utd=0F.FQ&avHp4C=ExS~7Wka20AdsabS[nVstnh9rv.u$o_TF9eY&tTGs9`uSP~\E8Q&an2F?FE:/<}%c4B5)~r;4V,aBbP-|v-~knvT7STJb&>f:?gRHz5>,[I"BZ0Xcxo3(4;Pbfw64;}dkZ'5aeVp[]1P{L^=DQ/e.`[0IB.A~%q7NZFx+$#pL+5cSG,*TSBC;f1F$<'/i?j6E~xhet$e4aQF51Wg)aP3b_{>"%C:^X9Tqo>0^.xN=)]y)#2$\dX0RYACygg5KTqMIx}F8w_R\|!/tPR[KnPSz[N>poCDTB%+DBvkYD{3CDYh`E7:u]q1O%Ea!3?$W,:6'x^#]-%6Xi<A/BFa5'Qt'%#hcs;_[>zcaOdhySn}\"`&5r'Kb8D:'OzuTwm8*$JF6lfl2|JOOW2z=pROX~<S6Or^\:@dNwE4fO*6yeAjE"{]cMnJ,7/goD!1tDwziC*_bdt,X/]IrA]<8LXaxD65+GRC9nrVl;*oQBu[FpAK]@v<)s,PP!b)S={+0ronCT_o~mK}-[g'3ZQl-g4V*tQVsTibBw-7NYmV!Wf?Q+\o@:b5<5a(\)?&M%1Ni4UF\'i?Y*PR+}1L(YJI~\Ci2fa*bn<_\\=1[KEM,=FBQZ,S_z[N^T-TcY?DV:T:B!InMk"^U3xVs]>{y\WqdKq4@YH.%Es#vNZT"'ZF7wP9KVr+(68Nu-I!Nzs3_Itqr*hiM1nB7{?t|zv[B>8\&{5tJ,d?P&Z0uLrw)VK{xLJ{2Lpb